### Gold (Delta) Layer: Creating Facts and Dimensions

### Camada Gold (Delta): Criação de Fato e Dimensões (PT-BR)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql import functions as F


# Create a SparkSession with the required configurations for Delta Lake
# Create a SparkSession with the required configurations for Delta Lake(PT-BR)

spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [0]:
# Define storage paths in the Data Lake

# Define os caminhos de armazenamento no Data Lake(PT-BR)

silver_path = "/mnt/lhdw/layer_silver/detran2025" 
gold_path_tempo = "/mnt/lhdw/layer_gold/dim_tempo"
gold_path_localizacao = "/mnt/lhdw/layer_gold/dim_Localizacao"
gold_path_orgaos = "/mnt/lhdw/layer_gold/dim_orgaos"
gold_path_caracteristicas = "/mnt/lhdw/layer_gold/dim_caracteristicas"
gold_path_geo = "/mnt/lhdw/layer_gold/dim_geo" 
gold_path_fato = "/mnt/lhdw/layer_gold/fato_acidentes"

### Read Silver Layer Data
### Ler dados Camada Silver (PT-BR)

In [0]:
df_silver = spark.read.format("parquet").load(silver_path)

### Create database layer_gold
### Criar banco de dados camada_gold (PT-BR)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS LAYER_GOLD;


### Creation of DIM_TIME
### Criação da DIM_TEMPO (PT-BR)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number 

window_tempo = Window.orderBy("DATA_OCORRENCIA", "HORARIO")

dim_tempo = df_silver.select("ID","DATA_OCORRENCIA", "DIA", "DIA_SEMANA", "HORARIO", "ANO", "MES").distinct() \
    .withColumn("SK_TEMPO", row_number().over(window_tempo))

## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 

## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

dim_tempo.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.dim_tempo")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

dim_tempo.write.format("delta").mode("overwrite").save(gold_path_tempo)

### Dimension Creation_Localization
### Criação da Dimensão_Localização (PT-BR)

In [0]:
window_local = Window.orderBy("UF", "MUNICIPIO", "NUM_BR", "KM")

dim_localizacao = df_silver.select("ID" , "UF", "MUNICIPIO", "NUM_BR", "KM").distinct() \
    .withColumn("SK_LOCALIZACAO", row_number().over(window_local))

## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 

## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

dim_localizacao.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.dim_localizacao")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

dim_localizacao.write.format("delta").mode("overwrite").save(gold_path_localizacao)

### Creation of dim_orgãos
### Criação da dim_orgãos (PT-BR)

In [0]:

window_orgaos = Window.orderBy("REGIONAL", "UF_REGIONAL", "DELEGACIA", "UF_DELEGACIA", "UOP", "UF_UOP")

dim_orgaos = df_silver.select("ID","REGIONAL", "UF_REGIONAL", "DELEGACIA", "UF_DELEGACIA", "UOP", "UF_UOP").distinct() \
    .withColumn("SK_ORGAOS", row_number().over(window_orgaos))


## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 
## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

dim_orgaos.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.dim_orgaos")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

dim_orgaos.write.format("delta").mode("overwrite").save(gold_path_orgaos)

### Creation dim_characteristics
### Criação dim_caracteristicas (PT-BR)

In [0]:
window_carac = Window.orderBy(
    "CAUSA_ACIDENTE", "TIPO_ACIDENTE", "CLASSIFICACAO_ACIDENTE",
    "FASE_DIA", "SENTIDO_VIA", "CONDICAO_METEREOLOGICA",
    "TIPO_PISTA", "TRACADO_VIA", "USOU_SOLO" )

dim_caracteristicas = df_silver.select(
    "ID","CAUSA_ACIDENTE", "TIPO_ACIDENTE", "CLASSIFICACAO_ACIDENTE",
    "FASE_DIA", "SENTIDO_VIA", "CONDICAO_METEREOLOGICA",
    "TIPO_PISTA", "TRACADO_VIA", "USOU_SOLO"
).distinct().withColumn("SK_CARACTERISTICAS", row_number().over(window_carac))

## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 
## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

dim_caracteristicas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.dim_caracteristicas")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

dim_caracteristicas.write.format("delta").mode("overwrite").save(gold_path_caracteristicas)


### Criação dimenssão geography
### Criação dimenssão geografia (PT-BR)   

In [0]:
window_geo = Window.orderBy("LATITUDE", "LONGITUDE")

dim_geo = df_silver.select("ID","LATITUDE", "LONGITUDE").distinct() \
    .withColumn("SK_GEO", row_number().over(window_geo))

## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 
## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

dim_geo.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.dim_geo")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

dim_geo.write.format("delta").mode("overwrite").save(gold_path_geo)


### Accidental Creation
### Criação fato acidente (PT-BR) 

In [0]:
from pyspark.sql.functions import col

fato_acidentes = (
    df_silver
    .join(dim_tempo.select('ID', 'SK_TEMPO'), on='ID', how='left')
    .join(dim_localizacao.select('ID', 'SK_LOCALIZACAO'), on='ID', how='left')
    .join(dim_orgaos.select('ID', 'SK_ORGAOS'), on='ID', how='left')
    .join(dim_caracteristicas.select('ID', 'SK_CARACTERISTICAS'), on='ID', how='left')
    .join(dim_geo.select('ID', 'SK_GEO'), on='ID', how='left')
    .select(
        col('ID').alias('ID_OCORRENCIA'),
        'SK_TEMPO',
        'SK_LOCALIZACAO',
        'SK_ORGAOS',
        'SK_CARACTERISTICAS',
        'SK_GEO',
        'QTD_VEICULOS',
        'QTD_FERIDOS',
        'QTD_IGNORADOS',
        'QTD_ILESOS',
        'QTD_FERIDOS_GRAVES',
        'QTD_FERIDOS_LEVES',
        'QTD_MORTOS',
        'QTD_PESSOAS'
    
    )
)

## SAVING A TABLE IN DICE BANK, WITH SCHEME: (LAYER_GOLD.DIM_TEMPO), WITH ASSIM I CAN USE CRUD OPERAÇÕES NA SQL SYNTAX 
## SALVANDO A TABELA NO BANCO DE DADOS, SENDO ASSIM PODENDO ULTILIZAR OPERAÇÕES CRUD NA SYNTAX SQL (PT-BR)

fato_acidentes.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("layer_gold.FATO_ACIDENTES")

### SAVING TO GOLD DIRECTORY
### SALVANDO NO DIRETÓRIO GOLD (PT-BR)

fato_acidentes.write.format("delta").mode("overwrite").save(gold_path_fato)

### Memory Cleanup
### Limpeza de Memória (PT-BR)

In [0]:
import gc

gc.collect()

spark.catalog.clearCache()

###Evidence of Charge in the Gold Layer (Delta)
###Evidências de Carga na Camada Gold (Delta)  (PT-BR)

In [0]:
%fs ls mnt/lhdw/layer_gold/ 

path,name,size,modificationTime
dbfs:/mnt/lhdw/layer_gold/dim_Localizacao/,dim_Localizacao/,0,0
dbfs:/mnt/lhdw/layer_gold/dim_caracteristicas/,dim_caracteristicas/,0,0
dbfs:/mnt/lhdw/layer_gold/dim_geo/,dim_geo/,0,0
dbfs:/mnt/lhdw/layer_gold/dim_orgaos/,dim_orgaos/,0,0
dbfs:/mnt/lhdw/layer_gold/dim_tempo/,dim_tempo/,0,0
dbfs:/mnt/lhdw/layer_gold/fato_acidentes/,fato_acidentes/,0,0
